# Medical vs Non-Medical Image Classifier
### TensorFlow/Keras Quick Training + PDF/URL Inference

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os, cv2

print("✅ TensorFlow version:", tf.__version__)


✅ TensorFlow version: 2.19.0


In [ ]:
conda install -c conda-forge tensorflow-datasets


In [ ]:
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
import requests, zipfile

# 📥 Download MedMNIST (Chest X-rays)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()  # Replace with MedMNIST loader

# 📥 Download Kaggle Natural Images dataset (placeholder)
url = "https://data.deepai.org/naturalimages.zip"
zip_path = "natural_images.zip"
if not os.path.exists(zip_path):
    r = requests.get(url)
    with open(zip_path, "wb") as f:
        f.write(r.content)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("natural_images")

# 🔄 Preprocess into (image, label) arrays
IMG_SIZE = 128
def preprocess(img):
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img / 255.0

# Combine MedMNIST (label=1) and Natural Images (label=0)
# ... load images into X, labels into y ...
# X = np.array([...]); y = np.array([...])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
import os, cv2, numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

IMG_SIZE = 128

# ✅ Preprocess helper
def preprocess(img):
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img / 255.0

# ✅ Placeholder MedMNIST data (using MNIST here as medical)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.stack([x_train]*3, axis=-1)
y_train = np.ones(len(x_train))  # medical label=1

# ✅ Load your local non-medical images from natural_images/
X_nonmed, y_nonmed = [], []
for root, dirs, files in os.walk("natural_images"):
    for file in files:
        img_path = os.path.join(root, file)
        img = cv2.imread(img_path)
        if img is not None:
            X_nonmed.append(preprocess(img))
            y_nonmed.append(0)  # non-medical label=0

X_nonmed = np.array(X_nonmed)
y_nonmed = np.array(y_nonmed)

# ✅ Combine datasets
X = np.concatenate([x_train[:500], X_nonmed])
y = np.concatenate([y_train[:500], y_nonmed])
X = np.array([preprocess(img) for img in X])

# ✅ Split train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print("✅ Dataset ready:", X_train.shape, X_val.shape)

# ✅ Build MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
preds = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=preds)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ✅ Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3, batch_size=32)
state=42)
print("✅ Dataset ready:", X_train.shape, X_val.shape)

# 🔨 Build MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
preds = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=preds)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ✅ Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3, batch_size=32)

SyntaxError: unmatched ')' (3607209687.py, line 58)